In [ ]:
from pickle import load
from utils.model import *
from utils.load_data import loadTrainData, loadValData, data_generator
from keras.callbacks import ModelCheckpoint
from config import config, rnnConfig
import random
# Setting random seed for reproducibility of results
random.seed(config['random_seed'])

"""
    *Some simple checking
"""
assert type(config['num_of_epochs']) is int, 'Please provide an integer value for `num_of_epochs` parameter in config.py file'
assert type(config['max_length']) is int, 'Please provide an integer value for `max_length` parameter in config.py file'
assert type(config['batch_size']) is int, 'Please provide an integer value for `batch_size` parameter in config.py file'
assert type(config['beam_search_k']) is int, 'Please provide an integer value for `beam_search_k` parameter in config.py file'
assert type(config['random_seed']) is int, 'Please provide an integer value for `random_seed` parameter in config.py file'
assert type(rnnConfig['embedding_size']) is int, 'Please provide an integer value for `embedding_size` parameter in config.py file'
assert type(rnnConfig['LSTM_units']) is int, 'Please provide an integer value for `LSTM_units` parameter in config.py file'
assert type(rnnConfig['dense_units']) is int, 'Please provide an integer value for `dense_units` parameter in config.py file'
assert type(rnnConfig['dropout']) is float, 'Please provide a float value for `dropout` parameter in config.py file'

"""
	*Load Data
	*X1 : Image features
	*X2 : Text features(Captions)
"""
X1train, X2train, max_length = loadTrainData(config)

X1val, X2val = loadValData(config)

"""
	*Load the tokenizer
"""
tokenizer = load(open(config['tokenizer_path'], 'rb'))
vocab_size = len(tokenizer.word_index) + 1

"""
	*Now that we have the image features from CNN model, we need to feed them to a RNN Model.
	*Define the RNN model
"""
# model = RNNModel(vocab_size, max_length, rnnConfig, config['model_type'])
model = AlternativeRNNModel(vocab_size, max_length, rnnConfig, config['model_type'])
print('RNN Model (Decoder) Summary : ')
print(model.summary())

"""
    *Train the model save after each epoch
"""
num_of_epochs = config['num_of_epochs']
batch_size = config['batch_size']
steps_train = len(X2train)//batch_size
if len(X2train)%batch_size!=0:
    steps_train = steps_train+1
steps_val = len(X2val)//batch_size
if len(X2val)%batch_size!=0:
    steps_val = steps_val+1
model_save_path = config['model_data_path']+"model_"+str(config['model_type'])+"_epoch-{epoch:02d}_train_loss-{loss:.4f}_val_loss-{val_loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(model_save_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks = [checkpoint]

print('steps_train: {}, steps_val: {}'.format(steps_train,steps_val))
print('Batch Size: {}'.format(batch_size))
print('Total Number of Epochs = {}'.format(num_of_epochs))

# Shuffle train data
ids_train = list(X2train.keys())
random.shuffle(ids_train)
X2train_shuffled = {_id: X2train[_id] for _id in ids_train}
X2train = X2train_shuffled

# Create the train data generator
# returns [[img_features, text_features], out_word]
generator_train = data_generator(X1train, X2train, tokenizer, max_length, batch_size, config['random_seed'])
# Create the validation data generator
# returns [[img_features, text_features], out_word]
generator_val = data_generator(X1val, X2val, tokenizer, max_length, batch_size, config['random_seed'])

# Fit for one epoch
model.fit_generator(generator_train,
            epochs=num_of_epochs,
            steps_per_epoch=steps_train,
            validation_data=generator_val,
            validation_steps=steps_val,
            callbacks=callbacks,
            verbose=1)

"""
	*Evaluate the model on validation data and ouput BLEU score
"""
print('Model trained successfully. Running model on validation set for calculating BLEU score using BEAM search with k={}'.format(config['beam_search_k']))

In [1]:
evaluate_model_beam_search(model, X1val, X2val, tokenizer, max_length, beam_index=config['beam_search_k'])


/home/ubuntu/anaconda3/envs/imgdesc_tf2/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


16:15:19: Using Inceptionv3 model
16:15:19: Image features already generated at model_data/features_inceptionv3.pkl
16:15:19: Parsed caption file already generated at model_data/captions.txt
16:15:19: Available images for training: 6000
16:15:19: Available captions for training: 30000
16:15:19: Available images for validation: 1000
16:15:19: Available captions for validation: 5000
RNN Model (Decoder) Summary : 
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 40)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 2048)         0                                            
_________________________________________________________________________

/home/ubuntu/anaconda3/envs/imgdesc_tf2/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
94/94 [==============================] - 107s 1s/step - loss: 5.6551 - val_loss: 5.1262

Epoch 00001: val_loss improved from inf to 5.12617, saving model to model_data/model_inceptionv3_epoch-01_train_loss-5.6562_val_loss-5.1262.hdf5
Epoch 2/20
94/94 [==============================] - 107s 1s/step - loss: 5.2239 - val_loss: 4.9026

Epoch 00002: val_loss improved from 5.12617 to 4.90257, saving model to model_data/model_inceptionv3_epoch-02_train_loss-5.2244_val_loss-4.9026.hdf5
Epoch 3/20
94/94 [==============================] - 108s 1s/step - loss: 4.9935 - val_loss: 4.6544

Epoch 00003: val_loss improved from 4.90257 to 4.65441, saving model to model_data/model_inceptionv3_epoch-03_train_loss-4.9941_val_loss-4.6544.hdf5
Epoch 4/20
94/94 [==============================] - 109s 1s/step - loss: 4.7146 - val_loss: 4.3356

Epoch 00004: val_loss improved from 4.65441 to 4.33565, saving model to model_data/model_inceptionv3_epoch-04_train_loss-4.7153_val_loss-4.3356.hdf5
Epoch 5/

  0%|          | 0/1000 [00:00<?, ?it/s]

Model trained successfully. Running model on validation set for calculating BLEU score using BEAM search with k=3


100%|██████████| 1000/1000 [1:02:45<00:00,  3.77s/it]


BLEU Scores :
A perfect match results in a score of 1.0, whereas a perfect mismatch results in a score of 0.0.
BLEU-1: 0.584653
BLEU-2: 0.346231
BLEU-3: 0.249837
BLEU-4: 0.133738
